In [1]:
import flymovie as fm
import numpy as np
import scipy.ndimage as ndi
%load_ext autoreload
%autoreload 2

In [15]:
def calc_nM(im_dims, brightness, kernel, sample_mean, sample_bg):
    # multiply each side length, 1e-27 is 1e-9 meters cubed to convert nm to m, 1000 is liters in a cubic meter
    L = np.product(im_dims) * 1e-27 * 1000 # in liters
    mols = 1 / 6.022e23
    M = mols / L
    nM_ref = M / 1e-9 # nM for 1 molecule per pixel

    im = ndi.convolve(np.ones((50,50,50)), kernel)
    mean_ref = im[25,25,25]

    nM = (sample_mean - sample_bg) / mean_ref * nM_ref
    return nM

### Add files and params

In [19]:
kernel_peak = 22_000
kernel_bg = 600
sample_mean = 4500
sample_bg = 300
n_gfps = 120
# 34 for egfp, 93 for mneon, 54 for sfgfp
brightness = 34
kernelfile = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/PSFs/psf_20220519_insitu_nm100x50x50_px27x25x25.pkl'
kernel_dims = (100, 50, 50)
im_dims = (100, 50, 50)


### Calculate kernel for 1 GFP.

In [20]:
kernel = fm.load_pickle(kernelfile)
kernel = ndi.zoom(kernel, np.divide(kernel_dims, im_dims), order=3)
kernel = (kernel - np.min(kernel)) / (np.max(kernel) - np.min(kernel))
kernel_1gfp = kernel * ((kernel_peak - kernel_bg) / np.max(kernel)) / n_gfps

### Calculate concentration

In [21]:
calc_nM(im_dims, brightness, kernel_1gfp, sample_mean, sample_bg)

173.14466932739143

2022-06-01

H2B: 2900 - 5800 nM  
Bicoid 53 nM - 166 nM  
Rpb1: 173 - 240
Zld: 

## Volumes:

In [38]:
im = fm.read_czi('/Volumes/stad3/2022-05-31/rpb1-measurement-1pct-vol-em2-01.czi', swapaxes=False)
fm.viewer(im)

interactive(children=(Dropdown(description='Color', index=10, options=('prism', 'plasma', 'Greens', 'gray', 'i…

In [39]:
sm = ndi.gaussian_filter(im, 4)
mask = np.where(sm > 2000, 1, 0)
lmask,_ = ndi.label(mask)
fm.viewer(lmask)

interactive(children=(Dropdown(description='Color', index=10, options=('prism', 'plasma', 'Greens', 'gray', 'i…

In [40]:
fm.viewer(np.where(lmask == 2, 1, 0))

interactive(children=(Dropdown(description='Color', index=10, options=('prism', 'plasma', 'Greens', 'gray', 'i…

In [41]:
np.mean(im[lmask == 2])

3265.5351922694203